#### Notes
- unzip address points gdb before running script
- process of elimination is used to wittle out parcels
- owned unit groupings (OUG) relies parcel common areas
    - if those aren't are accurately attributed, parcels within won't be correctly attributed
- when reusing scripts for later year, qaqc parcel ids used in queries


In [1]:
import arcpy
import os
import pandas as pd
from arcgis import GIS
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

# show all columns
pd.options.display.max_columns = None

In [2]:
# tag_expression and code block for tagging parcels
tag_expression = "createTag(!TAG!, '{}')"
tag_codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

In [3]:
# input taz shapefile and parcels
taz_shp = '.\\Inputs\\TAZ.shp'
parcels = '.\\Inputs\\Utah_Salt_Lake_County_Parcels_LIR.gdb\\Parcels_SaltLake_LIR_UTM12'


In [4]:
# create output gdb
outputs = '.\\Outputs'
gdb = os.path.join(outputs, "saltlake.gdb")
if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs, "saltlake.gdb")
    
scratch = os.path.join(outputs, "scratch.gdb")
if not arcpy.Exists(scratch):
    arcpy.CreateFileGDB_management(outputs, "scratch.gdb")

In [5]:
# get address points without base address point
address_pts_lyr = arcpy.MakeFeatureLayer_management('.\\Inputs\\AddressPoints.gdb\\AddressPoints','address_pts_lyr')
query = (''' PtType <> 'BASE ADDRESS' ''')
arcpy.SelectLayerByAttribute_management(address_pts_lyr, 'NEW_SELECTION', query)
address_pts_no_base = arcpy.FeatureClassToFeatureClass_conversion(address_pts_lyr, gdb, '_00_address_pts_no_base')

In [6]:
# select parcels within modeling area
parcels_layer = arcpy.MakeFeatureLayer_management(parcels, 'parcels') 
arcpy.SelectLayerByLocation_management(parcels_layer, 'HAVE_THEIR_CENTER_IN', taz_shp)
parcels_for_modeling = arcpy.FeatureClassToFeatureClass_conversion(parcels_layer, gdb, '_01_parcels_for_modeling')

# recalc acreage 
# arcpy.CalculateGeometryAttributes_management(parcels_for_modeling, [["PARCEL_ACRES", "AREA"]], area_unit='ACRES')
arcpy.CalculateField_management(parcels_for_modeling, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

# house count field is a string for some reason, so this is the fix
parcels_for_modeling_layer = arcpy.MakeFeatureLayer_management(parcels_for_modeling, 'parcels_for_modeling_lyr') 
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', """ HOUSE_CNT IS NULL """)
arcpy.CalculateField_management(parcels_for_modeling_layer, "HOUSE_CNT", """0""", "PYTHON3")
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
arcpy.AddField_management(parcels_for_modeling_layer, 'HOUSE_CNT2', 'LONG')
arcpy.CalculateField_management(parcels_for_modeling_layer, "HOUSE_CNT2", """int(!HOUSE_CNT!)""", "PYTHON3")

# # dissolve parcels by address to group together related parcels
# dissolved_parcels = arcpy.Dissolve_management(parcels_for_modeling_lyr, os.path.join(gdb, '_01_parcels_dissolve'), ["PARCEL_ADD"], 
#                           [['TOTAL_MKT_VALUE', 'SUM'],['LAND_MKT_VALUE', 'SUM'], ['PARCEL_ACRES', 'SUM'], ['BLDG_SQFT', 'SUM'], ['BUILT_YR', 'MAX'],
#                           ['BUILT_YR', 'SUM'], ['HOUSE_CNT2', 'SUM'],['PROP_TYPE', 'MIN'],['TAXEXEMPT_TYPE', 'MODE']])        
        

<Result 'parcels_for_modeling_lyr'>

In [7]:
# dp = pd.DataFrame.spatial.from_featureclass(dissolved_parcels)
# dp.columns = ['OBJECTID', 'PARCEL_ADD', 'TOTAL_MKT_VALUE', 'LAND_MKT_VALUE',
#        'PARCEL_ACRES', 'BLDG_SQFT', 'BUILT_YR', 'BUILT_YR',
#        'HOUSE_CNT', 'PROP_TYPE', 'SHAPE']

# dissolved_parcels = os.path.join(gdb,'_01_parcels_dissolve2')
# dp.spatial.to_featureclass(location=os.path.join(gdb,'_01_parcels_dissolve2'))

In [8]:
# add a tag field for parcel type
arcpy.AddField_management(parcels_for_modeling_layer, 'TAG', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'TYPE_WFRC', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'SUBTYPE_WFRC', 'TEXT')

# count parts and rings
arcpy.AddField_management(parcels_for_modeling_layer, 'PARTS', 'LONG')
arcpy.AddField_management(parcels_for_modeling_layer, 'RINGS', 'LONG')

fields = ["shape@", 'PARTS', 'RINGS']
with arcpy.da.UpdateCursor(parcels_for_modeling_layer, fields) as cursor:
    for row in cursor:
        shape = row[0]
        parts = shape.partCount
        rings = shape.boundary().partCount   
        row[1] = parts
        row[2] = rings
        cursor.updateRow(row)

# # create the master layer for selection
# parcels_for_modeling_layer = arcpy.MakeFeatureLayer_management(parcels_for_modeling_lyr, 'parcels_for_modeling_layer')
        
# delete parcels with empty geometry
query = (""" PARTS =  0""")
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# get a count of all parcels
count_all = arcpy.GetCount_management(parcels_for_modeling_layer)
print("# initial parcels in modeling area:\n {}".format(count_all))

# # create centroids for all parcels
# parcels_for_modeling_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
#                                                                  os.path.join(gdb, '_01b_parcels_for_modeling_centroids'), 
#                                                                  "INSIDE")     

# initial parcels in modeling area:
 384322


In [9]:
###########
# churches
###########

query = ("""PROP_TYPE IN ('956') OR TAXEXEMPT_TYPE IN ('ER') """)
tag="churches"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

1460 "churches" parcels were selected.
382862 parcels remain...


In [10]:
###############
# agriculture
###############

query = """ PROP_TYPE IN ('811','812','816','817', '818','830','850') """
tag="agriculture"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

1291 "agriculture" parcels were selected.
381571 parcels remain...


In [11]:
###############
# open space
###############

# get parcels on parks
parks = ".\\Inputs\\ParksLocal.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=parks,
                                       selection_type='NEW_SELECTION')

# get parcels on public lands that are not military
land_ownership_lyr = arcpy.MakeFeatureLayer_management('./Inputs/UT_SITLA_Ownership_LandOwnership_WM.shp','land_ownership_lyr') 

query= (""" OWNER <> 'Private' And DESIG <> 'Military' """)
arcpy.SelectLayerByAttribute_management(land_ownership_lyr, 'NEW_SELECTION', query)

arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=land_ownership_lyr,
                                       selection_type='ADD_TO_SELECTION')

# get cemetery parcels using overlay
cemeteries_lyr = arcpy.MakeFeatureLayer_management('.\\Inputs\\Cemeteries.shp','cemeteries_lyr') 
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="INTERSECT",
                                       select_features=cemeteries_lyr,
                                       selection_type='ADD_TO_SELECTION')

# get parcels that are golf courses using overlay
golf_courses = ".\\Inputs\\GolfCourses.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=golf_courses,
                                       selection_type='ADD_TO_SELECTION')

# query parcel attributes
query= (""" PROP_TYPE IN ('951','960','961') OR TAXEXEMPT_TYPE = 'CM' """)
tag = "open_space"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'ADD_TO_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

6806 "open_space" parcels were selected.
374765 parcels remain...


In [12]:
###############
# health care
###############

query = (""" PROP_TYPE in ('524', '527','547','560','561') """)
tag = 'healthcare'

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

511 "healthcare" parcels were selected.
374254 parcels remain...


In [13]:
###############
# education
###############

query = (""" PROP_TYPE IN  ('954','952','577') OR TAXEXEMPT_TYPE = 'EE' OR PARCEL_ID IN ('33123520020000', 
            '27251270410000', '21253850060000', '16091030030000') """)
tag="education"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

699 "education" parcels were selected.
373555 parcels remain...


In [14]:
###############
# government
###############

query = (""" PROP_TYPE IN ('570','953','955') OR PARCEL_ID IN ('28031050020000') """)
tag="government"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

5225 "government" parcels were selected.
368330 parcels remain...


In [15]:
###############
# retail
###############
query=(""" PROP_TYPE IN ('503','507','513','514','515','516','518','523','525','528','529','539','530','536', '537','548','549',
            '551','553','562','564','573','574','575','578','581','582','583','584','585','591','675','749','775','914') 
            OR PARCEL_ID IN ('16073130110000','16073130120000','16073130160000','16073130130000','16073130170000',
            '16073130140000','16073130180000','16073130150000','16073130190000', '26233270030000', '27134260150000', 
            '14241040100000', '22171020030000') """)


tag="retail"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

7393 "retail" parcels were selected.
360937 parcels remain...


In [16]:
###############
# industrial
###############

query = (""" (PROP_TYPE IN ('200','202','203', '550', '552', '556','695','795','915')) OR (PROP_CLASS = 'I') 
                OR (PARCEL_ID IN ('15173000540000', '16081760150000')) """)
tag="industrial"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

6768 "industrial" parcels were selected.
354169 parcels remain...


In [17]:
###############
# office
###############

query= ("""PROP_TYPE IN ('506','566','590', '660', '760','916') OR PARCEL_ID IN ('28271520010000') """)
tag="office"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

4864 "office" parcels were selected.
349305 parcels remain...


In [18]:
################
# single_family
################

query = (''' PROP_TYPE IN ('103','104','111', '118', '511', '888','994','998') OR PARCEL_ID IN ('28304780680000') ''')
tag="single_family"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# manual select some strays
query = (''' PARCEL_ID IN ('26253360020000','26253360050000','26253360080000','26253300160000','26253300180000',
'33072760080000','33072780070000','33102280070000','33102310150000') ''')
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'ADD_TO_SELECTION', query)

# single family - environmental degradation
query = (''' PROP_TYPE IN ('521') AND HOUSE_CNT2 > 0 ''')
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'ADD_TO_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
single_family = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))


#should count address points in resulting features and look for duplexes/mh

231898 "single_family" parcels were selected.
117407 parcels remain...


In [19]:
########################
# single_family w/ ADU
########################

query = (''' PROP_TYPE IN ('106') ''')
tag="single_family_adu"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'single_family_adu'",
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))



# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))


#should count address points in resulting features and look for duplexes/mh

1446 "single_family_adu" parcels were selected.
115961 parcels remain...


In [20]:
################
# duplexes
################

query = (''' PROP_TYPE IN ('112', '512') ''')
tag="duplex"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

5415 "duplex" parcels were selected.
110546 parcels remain...


In [21]:
################
# Residential Multi
################

query = (''' PROP_TYPE IN ('997') ''')
tag="residential_multi"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

532 "residential_multi" parcels were selected.
110014 parcels remain...


In [22]:
##########################
# Residential Salvage
##########################

# In disrepair, will likely need to be demolished
# Set the friction factor for these to be lower

query = (''' PROP_TYPE IN ('501', '993') ''')
tag="residential_salvage"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

79 "residential_salvage" parcels were selected.
109935 parcels remain...


In [23]:
###############
# utilities
###############

# select parcels using query
query = (''' PROP_TYPE IN ('950', '958') ''')
tag="utilities"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

13 "utilities" parcels were selected.
109922 parcels remain...


In [24]:
#################
# group quarters
#################

query= (""" PROP_TYPE IN ('105','540','576') """)
tag = "group_quarters"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

143 "group_quarters" parcels were selected.
109779 parcels remain...


In [25]:
######################
# Apartments
######################


query= (""" PROP_TYPE IN ('110','113','114', '115', '120', '150', '199') OR PARCEL_ID IN ('16294330010000', '16292380050000') """)
tag = "apartments"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="!PROP_TYPE!",
                                expression_type="PYTHON3")

# create the feature class for the parcel type
apt_commons = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_03_{}'.format(tag))


##############

query = """ PROP_TYPE IN ('504','713', '913') or PARCEL_ID IN ('16294330010000') """

# filter for common areas
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")


common_areas =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_03a_apt_commons')

# fill in holes via eliminate polygon part
apt_commons2 = arcpy.EliminatePolygonPart_management(common_areas, os.path.join(scratch, '_03b_apt_common_areas_filled'),
                                                      condition='PERCENT', part_area_percent=60, part_option='CONTAINED_ONLY')


#=========================
# get merge apt commons
#=========================

merged_apt_commons = arcpy.Merge_management([apt_commons, apt_commons2], 
                                              os.path.join(scratch, '_03c_merged_apt_commons'))
# recalc acreage 
# arcpy.CalculateGeometryAttributes_management(merged_apt_commons),"PARCEL_ACRES" "AREA",'', 'ACRES')   
arcpy.CalculateField_management(merged_apt_commons, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON", )

#==================================================
# summarize units attributes within common areas
#==================================================

# get parcels that are contained by apt common areas
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=merged_apt_commons, selection_type='NEW_SELECTION')

# convert condo parcels that are contained by common areas into centroids
apt_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
                                                  os.path.join(scratch, '_03d_apt_centroids'), "INSIDE")




# use spatial join to summarize market value & acreage
target_features = merged_apt_commons
join_features = apt_centroids
output_features = os.path.join(scratch, "_03e_apt_oug_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# # parcel acres
# fieldindex = fieldmappings.findFieldMapIndex('PARCEL_ACRES')
# fieldmap = fieldmappings.getFieldMap(fieldindex)
# fieldmap.mergeRule = 'Sum'
# fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "CONTAINS")

# rename join_count
arcpy.CalculateField_management(oug_sj, field='parcel_count', expression="!Join_Count!".format(tag),
                                expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj, "Join_Count")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = oug_sj 
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_apartment")

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           match_option="INTERSECTS")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!".format(tag), expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")



#=========================
# final summary
#=========================

# delete features from working parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=merged_apt_commons, selection_type='NEW_SELECTION')
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=merged_apt_commons, selection_type='ADD_TO_SELECTION')
# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# select all features within merged dataset and delete
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

5896 "apartments" parcels were selected.
103883 parcels remain...


In [26]:
################################
# OUG - Condos
################################

tag = "condo"

condo_groupings_reviewed = r'.\Inputs\ougs_reviewed.gdb\condos_reviewed2'


# recalc acreage
#arcpy.AddField_management(condo_groupings_reviewed, 'PARCEL_ACRES', 'FLOAT')
# arcpy.CalculateGeometryAttributes_management(condo_groupings_reviewed, [["PARCEL_ACRES", "AREA"]], area_unit='ACRES')
arcpy.CalculateField_management(condo_groupings_reviewed, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#==================================================
# summarize units attributes within common areas
#==================================================

# parcels that are contained by condo common areas
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=condo_groupings_reviewed, selection_type='NEW_SELECTION')

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# convert condo parcels that are contained by common areas into centroids
condo_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
                                                  os.path.join(scratch, '_04j_condos_centroids'), "INSIDE")



# use spatial join to summarize market value & acreage
target_features = condo_groupings_reviewed
join_features = condo_centroids
output_features = os.path.join(scratch, "_04k_condo_oug_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# # parcel acres
# fieldindex = fieldmappings.findFieldMapIndex('PARCEL_ACRES')
# fieldmap = fieldmappings.getFieldMap(fieldindex)
# fieldmap.mergeRule = 'Sum'
# fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# year
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mode'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)


# run the spatial join, use 'Join_Count' for number of units
oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "CONTAINS")

# rename join_count
arcpy.CalculateField_management(oug_sj, field='parcel_count', expression="!Join_Count!".format(tag),
                                expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj, "Join_Count")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = oug_sj 
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_condo")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECTS")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!".format(tag), expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

#################################
# WRAP-UP
#################################

# delete features from working parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=oug_sj2, selection_type='NEW_SELECTION')
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=oug_sj2, selection_type='ADD_TO_SELECTION')

count_type = arcpy.GetCount_management(parcels_for_modeling_layer)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

30029 "condo" parcels were selected.
73854 parcels remain...


In [27]:
#####################
# low income apartments
#####################

query= (""" PROP_TYPE IN ('142') """)
tag = "low_income_housing"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
features = arcpy.EliminatePolygonPart_management(parcels_for_modeling_layer, os.path.join(scratch, '_06a_{}'.format(tag)),
                                                      condition='PERCENT', part_area_percent=60, part_option='CONTAINED_ONLY')
layer = arcpy.MakeFeatureLayer_management(features, 'layer') 


# roundabout way to eliminate individual units inside low income common areas
# query= (""" TAXEXEMPT_TYPE = 'MC' """)
# arcpy.SelectLayerByAttribute_management(layer, 'NEW_SELECTION', query)
# common_areas = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_06b_{}_commons'.format(tag))


arcpy.SelectLayerByLocation_management(in_layer=layer,overlap_type="WITHIN",
                                       select_features=features,
                                       selection_type='NEW_SELECTION')
layer = arcpy.DeleteFeatures_management(layer)

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# recalc acreage 
# arcpy.CalculateGeometryAttributes_management(features, [["PARCEL_ACRES", "AREA"]], area_unit='ACRES')
arcpy.CalculateField_management(features, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = features
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_LowIncomeApts")

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           match_option="INTERSECTS")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!".format(tag), expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

225 "low_income_housing" parcels were selected.
73629 parcels remain...


In [28]:
#################
# OUG - mobile home parks
#################

tag="mobile_home"

# use overlay to select mobile home parks parcels
mobile_home_parks = ".\\Inputs\\Mobile_Home_Parks.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=mobile_home_parks,
                                       selection_type='NEW_SELECTION')
query= (""" PROP_TYPE IN ('160', '918') """)
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'ADD_TO_SELECTION', query)


# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
mhp = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_07a_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# recalc acreage 
# arcpy.CalculateGeometryAttributes_management(mhp, [["PARCEL_ACRES", "AREA"]], area_unit='ACRES')
arcpy.CalculateField_management(mhp, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = mhp
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_mobile_home_park")

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           match_option="INTERSECTS")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!".format(tag), expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")


# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))


64 "mobile_home" parcels were selected.
73565 parcels remain...


In [29]:
######################
# OUG - PUDs
######################

tag = "pud"

# #=====================
# # get condo commons 2
# #=====================
# query = """ PROP_TYPE IN ('119', '922') """
# arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
# puds =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_05a_pud_units')

# arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="INTERSECT",
#                                        select_features=puds,
#                                        selection_type='NEW_SELECTION')

# query = """ PROP_TYPE IN ('700', '701', '119', '922') """
# arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'SUBSET_SELECTION', query)
# common_areas =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_05b_pud_commons')
# common_areas_dissolved = arcpy.Dissolve_management(parcels_for_modeling_layer, 
#                                                    os.path.join(scratch, '_05c_pud_commons_dissolve'), 
#                                                    multi_part='SINGLE_PART')




# # parcels that are contained by pud common areas
# arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
#                                        select_features=common_areas_dissolved, selection_type='NEW_SELECTION')

# # calculate the type field
# arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
#                                 expression_type="PYTHON3")

# # convert pud parcels that are contained by common areas into centroids
# pud_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
#                                                   os.path.join(scratch, '_05c_pud_centroids'), "INSIDE")




puds_groupings_reviewed = r'.\Inputs\ougs_reviewed.gdb\puds_reviewed2'

# parcels that are contained by condo common areas
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=puds_groupings_reviewed, selection_type='NEW_SELECTION')

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# convert condo parcels that are contained by common areas into centroids
pud_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
                                                  os.path.join(scratch, '_04j_condos_centroids'), "INSIDE")


# recalc acreage
# arcpy.CalculateGeometryAttributes_management(puds_groupings_reviewed, [["PARCEL_ACRES", "AREA"]], area_unit='ACRES')
arcpy.CalculateField_management(puds_groupings_reviewed, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#==================================================
# summarize units attributes within pud areas
#==================================================

# use spatial join to summarize market value & acreage
target_features = puds_groupings_reviewed
join_features = pud_centroids
output_features = os.path.join(scratch, "_05d_pud_oug_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# # parcel acres
# fieldindex = fieldmappings.findFieldMapIndex('PARCEL_ACRES')
# fieldmap = fieldmappings.getFieldMap(fieldindex)
# fieldmap.mergeRule = 'Sum'
# fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# year
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mode'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "CONTAINS")

# calculate the type field
arcpy.CalculateField_management(oug_sj, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")


# rename join_count
arcpy.CalculateField_management(oug_sj, field='parcel_count', expression="!Join_Count!".format(tag),
                                expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj, "Join_Count")



#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = oug_sj
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_pud")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                                     fieldmappings, match_option="INTERSECTS")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!".format(tag), expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")


#################################
# WRAP-UP
#################################

# delete features from working parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=oug_sj2, selection_type='NEW_SELECTION')
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=oug_sj2, selection_type='ADD_TO_SELECTION')

count_type = arcpy.GetCount_management(parcels_for_modeling_layer)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)


# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

39471 "pud" parcels were selected.
34094 parcels remain...


In [30]:
#################
# commercial
#################

query= (""" PROP_TYPE IN ('500', '510', '520', '546','558','592', '593', '594', '595', '596') OR TAXEXEMPT_TYPE IN ('CC') 
            OR PROP_CLASS = 'C' """)
tag="commercial"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

4045 "commercial" parcels were selected.
30049 parcels remain...


In [31]:
#################
# roads
#################

query=""" (BLDG_SQFT = 0 And FLOORS_CNT = 0 And TOTAL_MKT_VALUE = 0 And PARCEL_ADD IS NULL) OR 
            PARCEL_ID IN ('16202760760000') """
tag="road"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

15535 "road" parcels were selected.
14514 parcels remain...


In [32]:
#################
# Vacant land
#################

query= (""" PROP_TYPE IN ('119','901','902','903', '905', '906', '908', '911', '917', '922') or (PROP_TYPE IS NULL) """)
tag = "vacant"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

5672 "vacant" parcels were selected.
8842 parcels remain...


In [33]:
#################
# Parking Structures
#################

query= (""" PROP_TYPE IN ('567', '904', '920') OR PARCEL_ID IN ('27201800030000') """)
tag = "parking_structure"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

16 "parking_structure" parcels were selected.
8826 parcels remain...


In [34]:
#################
# Undevelopable
#################

query= (""" PROP_TYPE IN ('711', '958', '999') OR TAXEXEMPT_TYPE IN ('IR', 'GB') OR PROP_TYPE = ' ' """)
tag = "undevelopable"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

#
query = (''' PROP_TYPE IN ('521') AND HOUSE_CNT2 < 1 ''')
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'ADD_TO_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

1616 "undevelopable" parcels were selected.
7210 parcels remain...


In [35]:
#################
# Related Parcel (SF)
#################

# select all related parcels
query= (""" PROP_TYPE IN ('957') AND PARCEL_ADD IS NOT NULL AND PARCEL_ADD <> ' '  """)
tag = "related_parcel"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
related_parcels = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_08_{}'.format(tag))

# merge related parcels into dc and dissolve if addresses match
merged = arcpy.Merge_management([single_family, related_parcels], os.path.join(scratch, '_08_merged_sf_related'))
merged_layer = arcpy.MakeFeatureLayer_management(merged, 'merged_layer')

# get parcels that have an address
query= (""" PARCEL_ADD IS NOT NULL AND PARCEL_ADD <> ' '  """)
arcpy.SelectLayerByAttribute_management(merged_layer, 'NEW_SELECTION', query)

# dissolve parcels that have addresses
dissolved_parcels = arcpy.Dissolve_management(merged_layer, os.path.join(scratch, '_08_merged_dissolve'), 
                                              ["PARCEL_ADD"], [['TOTAL_MKT_VALUE', 'SUM'],['LAND_MKT_VALUE', 'SUM'], 
                                              ['PARCEL_ACRES', 'SUM'], ['BLDG_SQFT', 'SUM'], ['BUILT_YR', 'MAX'],
                                              ['BUILT_YR', 'SUM'], ['HOUSE_CNT2', 'SUM'],['PROP_TYPE', 'MIN'], 
                                              ['TYPE_WFRC', 'FIRST']])

# invert selection and dissolve using OID
arcpy.SelectLayerByAttribute_management(merged_layer, 'SWITCH_SELECTION')
dissolved_parcels2 = arcpy.Dissolve_management(merged_layer, os.path.join(scratch, '_08_merged_dissolve2'), 
                                              ["PARCEL_ID"], [['TOTAL_MKT_VALUE', 'SUM'],['LAND_MKT_VALUE', 'SUM'], 
                                              ['PARCEL_ACRES', 'SUM'], ['BLDG_SQFT', 'SUM'], ['BUILT_YR', 'MAX'],
                                              ['BUILT_YR', 'SUM'], ['HOUSE_CNT2', 'SUM'],['PROP_TYPE', 'MIN'], 
                                              ['TYPE_WFRC', 'FIRST']])

merged2 = arcpy.Merge_management([dissolved_parcels, dissolved_parcels2], os.path.join(gdb, '_02_single_family_with_rp'))
merged2_layer = arcpy.MakeFeatureLayer_management(merged2, 'merged2_layer') 

query= (""" FIRST_TYPE_WFRC IN ('related_parcel') """)
arcpy.SelectLayerByAttribute_management(merged2_layer, 'NEW_SELECTION', query)
merged2_layer = arcpy.DeleteFeatures_management(merged2_layer)

# remove parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=merged2_layer, selection_type='NEW_SELECTION')
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

4464 "related_parcel" parcels were selected.
2746 parcels remain...


In [36]:
############################
# Community Land Trust (SF)
############################

# select all 651 parcels
query= (""" PROP_TYPE IN ('651') AND PARCEL_ADD IS NOT NULL AND PARCEL_ADD <> ' '  """)
tag = "comm_land_trust"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
clt = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_08_{}'.format(tag))

# dissolve parcels that have addresses
dissolved_parcels = arcpy.Dissolve_management(clt, os.path.join(gdb, '_02_{}'.format(tag)), 
                                              ["PARCEL_ADD"], [['TOTAL_MKT_VALUE', 'SUM'],['LAND_MKT_VALUE', 'SUM'], 
                                              ['PARCEL_ACRES', 'SUM'], ['BLDG_SQFT', 'SUM'], ['BUILT_YR', 'MAX'],
                                              ['BUILT_YR', 'SUM'], ['HOUSE_CNT2', 'SUM'],['PROP_TYPE', 'MIN'], 
                                              ['TYPE_WFRC', 'FIRST']])

dp_layer = arcpy.MakeFeatureLayer_management(dissolved_parcels, 'dp_layer') 


# remove parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=dp_layer, selection_type='NEW_SELECTION')
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

17 "comm_land_trust" parcels were selected.
2729 parcels remain...


In [37]:
# export remaining parcels
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_09_unclassed_parcels')

<Result '.\\Outputs\\saltlake.gdb\\_09_unclassed_parcels'>